# Features generation

This script generate the basic features include
1. mean encoding 
2. lag variable w.r.t last month
3. text data

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import gc

In [2]:
transactions    = pd.read_csv('../data/sales_train.csv.gz')
items           = pd.read_csv('../data/items.csv')
item_categories = pd.read_csv('../data/item_categories.csv')
shops           = pd.read_csv('../data/shops.csv')
tests           = pd.read_csv('../data/test.csv.gz')

In [3]:
## date 
trans_date = pd.to_datetime(transactions.date,format='%d.%m.%Y')
transactions['date'] = trans_date
transactions.set_index('date',inplace=True)
tests['date_block_num'] = 34

In [4]:
transactions['amount_price'] = transactions.item_price * transactions.item_cnt_day

In [5]:
##  outlier
cond = (transactions.item_price<100000) & (transactions.item_cnt_day <= 1000)
trans_wo_outlier = transactions[cond]

In [6]:
trans_wo_outlier.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,amount_price
date,,,,,,
2013-01-02,0,59,22154,999.00,1.0,999.00
2013-01-03,0,25,2552,899.00,1.0,899.00
2013-01-05,0,25,2552,899.00,-1.0,-899.00
2013-01-06,0,25,2554,1709.05,1.0,1709.05
2013-01-15,0,25,2555,1099.00,1.0,1099.00


* merge test with train

In [7]:
trans_wo_outlier = pd.concat([trans_wo_outlier, tests])
trans_wo_outlier = trans_wo_outlier.merge(items,on='item_id',how='left') ## add item name, item_Cat_id

In [8]:
trans_wo_outlier[trans_wo_outlier.date_block_num==34].head()

,ID,amount_price,date_block_num,item_cnt_day,item_id,item_price,shop_id,item_name,item_category_id
2935847,0.0,NaN,34,NaN,5037,NaN,5,"NHL 15 [PS3, русские субтитры]",19
2935848,1.0,NaN,34,NaN,5320,NaN,5,ONE DIRECTION Made In The A.M.,55
2935849,2.0,NaN,34,NaN,5233,NaN,5,"Need for Speed Rivals (Essentials) [PS3, русск...",19
2935850,3.0,NaN,34,NaN,5232,NaN,5,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23
2935851,4.0,NaN,34,NaN,5268,NaN,5,"Need for Speed [PS4, русская версия]",20


## Grid
Created a dataframe of all `Date_block_num`, `Store` and `Item` combinations:

In [9]:
trans_wo_outlier.columns

Index(['ID', 'amount_price', 'date_block_num', 'item_cnt_day', 'item_id',
       'item_price', 'shop_id', 'item_name', 'item_category_id'],
      dtype='object')

In [10]:
from itertools import product 
index_cols = ['shop_id','item_id','date_block_num']
grid = []
for block_num in trans_wo_outlier.date_block_num.unique():
    cur_shops = trans_wo_outlier.loc[trans_wo_outlier.date_block_num == block_num, 'shop_id'].unique()
    cur_items = trans_wo_outlier.loc[trans_wo_outlier.date_block_num == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]]))))


grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32)

In [11]:
grid.head()

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


In [12]:
grid.shape

(11128004, 3)

## Create mean encoding

In [13]:
trans_wo_outlier.head().T

,0,1,2,3,4
ID,NaN,NaN,NaN,NaN,NaN
amount_price,999,899,-899,1709.05,1099
date_block_num,0,0,0,0,0
item_cnt_day,1,1,-1,1,1
item_id,22154,2552,2552,2554,2555
item_price,999,899,899,1709.05,1099
shop_id,59,25,25,25,25
item_name,ЯВЛЕНИЕ 2012 (BD),DEEP PURPLE The House Of Blue Light LP,DEEP PURPLE The House Of Blue Light LP,DEEP PURPLE Who Do You Think We Are LP,DEEP PURPLE 30 Very Best Of 2CD (Фирм.)
item_category_id,37,58,58,58,56


For a shopid,itemid avg `item_price`, and `item_cnt` (sold) per month, 

In [14]:
trans_m = trans_wo_outlier.groupby(['date_block_num','shop_id','item_id']).agg(
    {'item_cnt_day':sum, 'item_price':np.mean}
).reset_index()
trans_m.rename(columns={'item_cnt_day':'item_cnt_month'},inplace=True)

trans_m = pd.merge(grid,trans_m,on=['date_block_num','shop_id','item_id'],how='left').fillna(0) 


In [15]:
## join with items category id/item_name, 
trans_m = trans_m.merge(items,on='item_id',how='left')

In [16]:
trans_m[trans_m.date_block_num==34].head()

,shop_id,item_id,date_block_num,item_cnt_month,item_price,item_name,item_category_id
10913804,5,5037,34,0.0,0.0,"NHL 15 [PS3, русские субтитры]",19
10913805,5,5320,34,0.0,0.0,ONE DIRECTION Made In The A.M.,55
10913806,5,5233,34,0.0,0.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19
10913807,5,5232,34,0.0,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23
10913808,5,5268,34,0.0,0.0,"Need for Speed [PS4, русская версия]",20


In [17]:
trans_wo_outlier.groupby(['item_id','date_block_num']).agg(np.mean).reset_index()[['item_price', 'item_id','date_block_num']].head()

,item_price,item_id,date_block_num
0,58.0,0,20
1,4490.0,1,15
2,4490.0,1,18
3,4490.0,1,19
4,4490.0,1,20


For a type_id(`item_id`) we encode aggtype(`avg`) of column_id(`item_price`) per month

We have 3 type_id: 
 - `item_id`,`shop_id`,`item_category_id`
 
3 ways of combine column_id:
 - `mean` of `item_price`, `sum` of `item_cnt`, `mean` of `item_cnt`

In [18]:
for type_id in ['item_id','shop_id','item_category_id']:
    for column_id, aggregator, aggtype in [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]:
        mean_df = trans_wo_outlier.groupby([type_id,'date_block_num']).aggregate(aggregator).reset_index()[[column_id,type_id,'date_block_num']]
        mean_df.columns = [type_id+'_'+aggtype+'_'+column_id, type_id, 'date_block_num']
        print('{}_{}_{}'.format(type_id, aggtype, column_id), end='\t')
        trans_m = pd.merge(trans_m,mean_df,on=['date_block_num',type_id],how='left')
        print('merge complete...')

item_id_avg_item_price	merge complete...
item_id_sum_item_cnt_day	merge complete...
item_id_avg_item_cnt_day	merge complete...
shop_id_avg_item_price	merge complete...
shop_id_sum_item_cnt_day	merge complete...
shop_id_avg_item_cnt_day	merge complete...
item_category_id_avg_item_price	merge complete...
item_category_id_sum_item_cnt_day	merge complete...
item_category_id_avg_item_cnt_day	merge complete...


## Lag variables

In [19]:
trans_m.columns[7:]

Index(['item_id_avg_item_price', 'item_id_sum_item_cnt_day',
       'item_id_avg_item_cnt_day', 'shop_id_avg_item_price',
       'shop_id_sum_item_cnt_day', 'shop_id_avg_item_cnt_day',
       'item_category_id_avg_item_price', 'item_category_id_sum_item_cnt_day',
       'item_category_id_avg_item_cnt_day'],
      dtype='object')

In [20]:
lag_variables  = list(trans_m.columns[7:])+['item_cnt_month']
lag_df = [trans_m] # 
lags = [1,2,3,5,12]
for lag in lags:
    print('lag:{} processing...'.format(lag))
    trans_new_df = trans_m.copy()
    trans_new_df.date_block_num += lag
    trans_new_df = trans_new_df[['date_block_num','shop_id','item_id']+lag_variables]
    trans_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    lag_df.append(trans_new_df)
    del trans_new_df 
    gc.collect()
#     trans_mean = pd.merge(trans_m, trans_new_df,on=['date_block_num','shop_id','item_id'] ,how='left') ###

lag:1 processing...
lag:2 processing...
lag:3 processing...
lag:5 processing...
lag:12 processing...


In [21]:
### stack it(less memory useage with gc)
sizeof_lagdf = len(lag_df)
for idx in range(sizeof_lagdf):
    if idx == 0:
        trans_final = lag_df.pop(0)
    else:
        lagdf_ele = lag_df.pop(0)
        trans_final = pd.merge(trans_final,lagdf_ele, on=['date_block_num','shop_id','item_id'],how='left',copy=False)
    print('idx:{}'.format(idx))

idx:0
idx:1
idx:2
idx:3
idx:4
idx:5


In [22]:
trans_final.shape

(11128004, 66)

In [23]:
gc.collect()

1064

## Extract text features

In [47]:
X_text_feats = trans_final[['date_block_num','shop_id','item_id',]].merge(shops,how='left',on='shop_id')
X_text_feats = X_text_feats.merge(items,how='left',on='item_id')
X_text_feats = X_text_feats.merge(item_categories,how='left',on='item_category_id')
gc.collect()

42

In [48]:
col_for_text = [
    'date_block_num',
    'item_name',
    'item_category_name',
    'shop_name'
]

In [49]:
X_text_feats[col_for_text].head()

,date_block_num,item_name,item_category_name,shop_name
0,0,ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,0,DEEP PURPLE The House Of Blue Light LP,Музыка - Винил,"Ярославль ТЦ ""Альтаир"""
2,0,DEEP PURPLE Who Do You Think We Are LP,Музыка - Винил,"Ярославль ТЦ ""Альтаир"""
3,0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),Музыка - CD фирменного производства,"Ярославль ТЦ ""Альтаир"""
4,0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,Музыка - Музыкальное видео,"Ярославль ТЦ ""Альтаир"""


# Storage 
- text

In [50]:
test_mask = X_text_feats['date_block_num']==34
cv_mask = X_text_feats['date_block_num']==33
train_mask = X_text_feats['date_block_num'] < 33

In [ ]:
## save it to h5 
text_store = pd.HDFStore('../data/feat/text_feats.h5')
text_store['X_text_feats_test'] = X_text_feats[col_for_text][test_mask]
text_store['X_text_feats_train'] = X_text_feats[col_for_text][train_mask]
text_store['X_text_feats_cv'] = X_text_feats[col_for_text][cv_mask]
text_store.close()

- for numerical features

In [24]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [25]:
col_to_drop  = [
    'item_cnt_month','item_price','item_name',
    'item_id_avg_item_price','item_id_sum_item_cnt_day','item_id_avg_item_cnt_day',
    'shop_id_avg_item_price','shop_id_sum_item_cnt_day','shop_id_avg_item_cnt_day', 
    'item_category_id_avg_item_price','item_category_id_sum_item_cnt_day','item_category_id_avg_item_cnt_day'
]

trans_df = trans_final.drop(col_to_drop,axis=1) ### downcast to float32, int32
trans_df = downcast_dtypes(trans_df) 

In [27]:
X_test = trans_df[trans_df['date_block_num']==34]
X_train = trans_df[trans_df['date_block_num']<33]
X_cv = trans_df[trans_df['date_block_num']==33]

y_train = trans_final[trans_final['date_block_num']<33].item_cnt_month.astype(np.int32)
y_cv = trans_final[trans_final['date_block_num']==33].item_cnt_month.astype(np.int32)

In [29]:
X_test.dtypes

shop_id                                       int32
item_id                                       int32
date_block_num                                int32
item_category_id                              int32
item_id_avg_item_price_lag_1                float32
item_id_sum_item_cnt_day_lag_1              float32
item_id_avg_item_cnt_day_lag_1              float32
shop_id_avg_item_price_lag_1                float32
shop_id_sum_item_cnt_day_lag_1              float32
shop_id_avg_item_cnt_day_lag_1              float32
item_category_id_avg_item_price_lag_1       float32
item_category_id_sum_item_cnt_day_lag_1     float32
item_category_id_avg_item_cnt_day_lag_1     float32
item_cnt_month_lag_1                        float32
item_id_avg_item_price_lag_2                float32
item_id_sum_item_cnt_day_lag_2              float32
item_id_avg_item_cnt_day_lag_2              float32
shop_id_avg_item_price_lag_2                float32
shop_id_sum_item_cnt_day_lag_2              float32
shop_id_avg_

In [30]:
store = pd.HDFStore('../data/feat/data.h5')
store['X_test'] = X_test
store['X_train'] = X_train
store['X_cv'] = X_cv
store['y_train'] = y_train
store['y_cv'] = y_cv
store.close()